## Datascience hackathon at UC Berkeley
### Project: Suggestion #1 Cleaning data sets and providing feedback
09/15/18
Spencer Harris
spharris@sas.upenn.edu

# This notebook is a draft concept
The author asks that any readers assume nothing about validity of statements made or code, for now

__Initial takeway from Project 1 explaination by R. Maestu:__

_The way the data is collected is a Excel spreadsheet with 300 columns, structure is very ungainly or show how can we use it?--
Fields can be have incorrect field type_
________

#1: Understanding the data gaps: visualizing the quantity & quality of existing data
Rafael Maestu

Every year, public water systems are required to submit an annual report that includes information such as water quantity used, sources of water, water fees charged, water quality violations, etc.

This reporting is now electronic through a webform, but data is not always usable (e.g., someone enters in a text string instead of a number) or complete (fields may be blank). The focus has been on collecting the annual reports but time and resources limits the ability to correct data gaps. As a result, the use of the data reported is limited.  

1) How might we redesign or shape paths for effective data collection and analysis for this required water quality reporting?



2) This project will work to visualize the problem, making a case for change. What percentage of records could be considered complete or accurate?



3) What system checks can we include to minimize data gaps? 



Relevant data sets: 
300 rows for a recent year published: 
Recommended Datasets List: #5, #6, #8

### Dataset #8
[Public System Water Produced Reported in the Electronic Annual Report (EAR)* __data location__(dataset #8) ](https://data.ca.gov/dataset/drinking-water-%E2%80%93-public-water-system-annually-reported-water-production-and-delivery-3)

Data Description: This is the 2016 EAR data that is self reported and includes more than produced and delivered water data...from the PWS (public system water) data submitted in their annual inventory information using the DRINC (Drinking water information clearninghouse) [Portal](https://drinc.ca.gov/drinc/) - EAR (electronic annual report) module - for the years 2013 through 2016.  

This data in the EAR is self-reported by PWS. 

goals: 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt

In [2]:
%pwd

'/home/lososian/projects/UC_water_datahackaton'

In [15]:
raw_df = pd.read_excel('./EAR_Datasets_1.xlsx', sheet_name='Table1_Final')

In [16]:
raw_df.shape

(6817, 87)

In [17]:
df_1 = raw_df
df_1.columns = df_1.columns.str.replace("[(.)]", "_") ## remove periods from column headers
df_1.columns = df_1.columns.str.replace("[( )]", "_") ## remove spaces from column headers
df_1.columns

Index(['PWSID', 'Water_System_Name', 'Water_System_Classification',
       'Water_System_Ownership', 'Physical_location_Address_1',
       'Physical_location_Address_2', 'Physical_location_City',
       'Physical_location_Zip', 'Office_Phone', 'Website_URL', 'Reporter_Name',
       'Reporter_Title', 'Reporter_Business_Phone', 'Reporter_Email',
       'Population', 'Population_Estimation_Method',
       'Population_Estimation_Method_Other', 'Population_Seasonal_Max',
       'Season_Begin_MM', 'Season_Begin_DD', 'Season_End_MM', 'Season_End_DD',
       'Population_Res_Begin_Date_MM', 'Population_Res_Begin_Date_DD',
       'Population_Res_End_Date_MM', 'Population_Res_End_Date_DD',
       'Population_Trans', 'Population_Trans_Begin_Date_MM',
       'Population_Trans_Begin_Date_DD', 'Population_Trans_End_Date_MM',
       'Population_Trans_End_Date_DD', 'Population_Non-Trans',
       'Population_Non-Trans_Begin_Date_MM',
       'Population_Non-Trans_Begin_Date_DD',
       'Population_Non-Tr

In [19]:
len(df_1['PWSID'][df_1['Water_System_Name'].notnull()].unique()) ##matches shape of df_1, gucci, can work with this

6813

In [20]:
len(df_1['PWSID'][df_1['Water_System_Classification'].notnull()].unique()) ## doesnt matche shape of df_1 .....
## looks like there is no option for a non-public water system option for the Water_System_Classification entry

6058

In [23]:
## rows with blank entries in Water_System_Classification
len(df_1.loc[(df_1.Water_System_Classification == ' ') ]) 

731

In [24]:
## rows with blank entries in Water_System_Classification and listed as privately owned business
len(df_1.loc[(df_1.Water_System_Classification == ' ') &
         (df_1.Water_System_Ownership == 'Privately owned business (non-community)')])

418

In [31]:
raw_df.loc[raw_df['Water_System_Classification'].str.contains('non public', case = False)] 
## problem of empty entires

ValueError: cannot index with vector containing NA / NaN values

In [30]:
raw_df['Water_System_Classification'].unique()

array(['Community Water System', ' ',
       'Transient Noncommunity Water System',
       'Nontransient Noncommunity Water System', nan,
       'Privately owned, PUC-regulated, for profit water company',
       'Local Government'], dtype=object)

# Issue 1
### public vs. non public distinction is not clear.

There are no entries in the __Water\_System\_Name__ column with "_non public_" as an entry, There should be? based on the explaination below [Source](https://data.ca.gov/sites/default/files/PRODUCTION%20POTABLE%20WATER%20DATA%20DICTIONARY.pdf):

"Water systems are classified
according to rules developed by the
U.S. Environmental Protection
Agency (EPA) and each state.
Water Systems fall into two broad
categories: public and non-public."

"A public water system can be further classified as one of
the following:  
__C__ - Community Serves at least 15 service connections
used by year-round residents or regularly serves 25 yearround
residents.  
__NTNC__ - Non-Transient Non-Community Serves at least
the same 25 non-residential individuals during 6 months of
the year.  
__NC__ - Transient Non-Community Regularly serves at least
25 non-residential individuals (transient) during 60 or more
days per year.  

This classification is based on information in each of the
following five areas:
1. Quantity and type of population served (e.g., transient,
residential).
2. Number of days served (or annual operating period
(i.e., the period(s) of the year that the water system is in
operation)).
3. Number of service connections (i.e., the number of
connections for each service area such as commercial,
agricultural, residential).
4. Activity status of the water system (i.e., active or
inactive).
5. Sources of water (e.g., reservoir, well. Sources of
water may be active or inactive depending on whether the
source is seasonal or year-round. The current date
determines active or inactive status). "

------

more explaination below from different [Source](http://drinc.ca.gov/ear/2016SWSHelp.htm#WSC):  

"PUBLIC WATER SYSTEM CLASSIFICATION	DESCRIPTION

__Community Water System__	A public water system that serves at least 15 service connections used by yearlong residents or regularly serves at least 25 yearlong residents of the area served by the system. The term ‘residence’ generally means single-family homes, but also includes dwelling units that are more or less equivalent to a residential home. This would include cabins, cottages, mobile homes, efficiency living units, apartments, etc., that are used on a long-term basis by residents. The term ‘yearlong’ infers use that exceeds 6 months of the year. Using this criterion, examples of community water systems may include second home subdivisions, cabin clusters, apartment buildings, mobile home or trailer parks, labor camps and correctional facilities.  

__Transient Noncommunity Water System__	A public water system that regularly serves at least 25 persons daily for at least 60 days out of the year, but does not serve the same 25 persons for over 6 months per year. The days do not have to be consecutive. Examples of a transient noncommunity water system include day-use facilities, campgrounds, resorts, rest-stops, restaurants, hotels, visitor centers, churches, cabin clusters that do not have access for part of the year, and businesses with fewer than 25 employees but regularly has over 25 customers daily for over 60 days per year.  

__Nontransient Noncommunity Water System__	A public water system that is not a community water system and that regularly serves at least 25 of the same persons over 6 months per year. Examples include businesses with over 25 employees, day care facilities, and schools,"

------------
## Suggestion

In the data set these are the values found in the __Water System Classification__ field:  
`Community Water System', ' ', 'Transient Noncommunity Water System', 'Nontransient Noncommunity Water System', nan, 'Privately owned, PUC-regulated, for profit water company', 'Local Government'`

Looking at the PDF submission [form](https://data.ca.gov/sites/default/files/2016EAR_SWSTemplate.pdf) This looks like an open field? it should be constrained to a Valid Value List via dropdown menu containing only:

__Community Water System__  
__Transient Noncommunity Water System__  
__Nontransient Noncommunity Water System__  
?__Non Public__? <-- suggestion


In [ ]:
## attempting to fill data gaps in Water_System_Classfication
df_1.loc[(df_1.Water_System_Classification == ' ') & 
          (df_1.Water_System_Ownership == 'Privately owned business (non-community)'),
         'Water_System_Classification'] = 'Non Public'

df_1.loc[(df_1.Water_System_Classification == ' ') & 
          (df_1.Water_System_Ownership.str.contains('Community Water System')),
         'Water_System_Classification'] = 'Community Water System'

df_1.loc[(df_1.Water_System_Classification == ' ') & 
          (df_1.Water_System_Name.str.contains('SCHOOL')),
         'Water_System_Classification'] = 'Nontransient Noncommunity Water System'

df_1.loc[(df_1.Water_System_Classification == ' ') & 
          (df_1.Water_System_Name.str.contains('CAMP')),
         'Water_System_Classification'] = 'Transient Noncommunity Water System'

# from here down is old/scrap/scratch, 9:50 AM 20180915

In [ ]:
df_1.loc[(df_1.Water_System_Classification == ' ') ]

In [ ]:
df_1

In [ ]:
len(df_1['PWSID'][df_1['Water_System_Classification'].notnull()].unique()) ## doesnt matche shape of df_1 .....

In [ ]:
df_1.loc[df_1.Water_System_Ownership == 'Privately owned business (non-community)' &
         df_1.Water_System_Classification.isnull(), 'Water_System_Classification'] = 'Non Public'

In [ ]:
if df_1['Water_System_Ownership'] == 'Privately owned business (non-community)':
    df_1['Water_System_Classification'] = 'Non Public'

### population (dataset #5), water required per person (dataset #5), water produced in house, water purchased,??? 

In [ ]:
raw_df_data5.columns

In [ ]:
len(raw_df_data5['Water.System.Name'].unique())

In [ ]:
type(raw_df_data5['Water.System.Name'][1])

In [ ]:
raw_df_data5.columns = raw_df_data5.columns.str.replace("[(.)]", "_") ## remove periods from column headers
raw_df_data5.columns = raw_df_data5.columns.str.replace("[( )]", "_") ## remove periods from column headers

In [ ]:
raw_df_data5

In [ ]:
#raw_df_data5.loc[raw_df_data5['_Population_Of_Service_Area_'] == raw_df_data5['_Population_Of_Service_Area_'].max()]

In [ ]:
raw_df_data5['_PWSID_'][raw_df_data5['Water_System_Name'].isnull()].unique()

In [ ]:
## % of unique (with respect to PWSID) entires which do not have population data
(len(raw_df_data5['_PWSID_'][raw_df_data5['_Population_Of_Service_Area_'].isnull()].unique())/
len(raw_df_data5['_PWSID_'][raw_df_data5['_Population_Of_Service_Area_'].notnull()].unique()) * 100)

In [ ]:
dict_PWSID_Water_System_Name = raw_df_data5.set_index('_PWSID_').to_dict()['Water_System_Name']

In [ ]:
## filling all 'Nan' in the Water_System_Name
raw_df_data5['Water_System_Name'] = raw_df_data5['Water_System_Name'].fillna(
    raw_df_data5._PWSID_.map(dict_PWSID_Water_System_Name))

In [ ]:
raw_df_data5['_PWSID_'][raw_df_data5['Water_System_Name'].isnull()].unique() ## seems legit

## fixing water classification system

In [ ]:
raw_df_data5['_PWSID_'][raw_df_data5['Water_System_Classification'].isnull()].unique()

In [ ]:


for i in raw_df_data5.columns:
    dict_%s = raw_df_data5.set_index('_PWSID_').to_dict()['i']
    